# Rearchitecting LLMs
## Surgical Optimization for Hyper-Efficient Models

### Chapter 5: Width Pruning
### Notebook: 03. OptiPFair Specialized Pruning
by [Pere Martra](https://github.com/peremartra)

In this notebook, we create a model specialized for the SMS Spam dataset using `optipfair`.

We will:
1.  Analyze layer importance using Cosine Similarity on the SMS dataset.
2.  Identify the 6 most important layers.
3.  Perform width pruning: Keep the 6 most important layers intact, and prune 40% of the neurons in the remaining layers.
4.  Compare the results with the standard pruning approach from Notebook 02.

In [ ]:
!pip install -q \
      "torch" \
      "transformers==4.55.4" \
      "accelerate==1.10.1" \
      "lm_eval==0.4.9.1" \
      "sentencepiece==0.2.1" \
      "datasets" \
      "langdetect"\
      "optipfair==0.2.1"

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from torch import nn
import copy
import gc
import time
import matplotlib.pyplot as plt
import optipfair
from optipfair import analyze_layer_importance
from optipfair.pruning.mlp_glu import compute_neuron_pair_importance_maw

In [ ]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/Rearchitecting-LLMs/main/utils.py

from utils import (
  evaluate_metrics, # Loss & Perpelexity
  generate_text, #test inference model
  clear_gpu_cache
)

In [ ]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Configuration

In [ ]:
MODEL_NAME = 'meta-llama/Llama-3.2-1B'
RECOVERY_SAMPLES = 1000
MAX_LENGTH = 512
BATCH_SIZE = 4
PRUNE_PERCENT_REST = 0.4  # Prune 40% of neurons in non-critical layers
EXPANSION_DIVISOR = 128

## Load Model and Datasets

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

In [ ]:
datasms = load_dataset('sms_spam', split=f'train[:{RECOVERY_SAMPLES}]')
print(f"SMS samples: {len(datasms)}")

In [ ]:
def prepare_dataset(dataset, text_field='text'):
    def tokenize_function(examples):
        if text_field in examples:
            texts = examples[text_field]
        elif 'sms' in examples:  # SMS dataset specific
            texts = examples['sms']
        else:
            texts = examples[list(examples.keys())[0]]

        return tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=MAX_LENGTH,
            return_tensors='pt'
        )

    tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    return DataLoader(tokenized, batch_size=BATCH_SIZE, shuffle=False)

dataloadersms = prepare_dataset(datasms)

## Layer Importance Analysis

We use `optipfair.analyze_layer_importance` to determine layer importance. This function typically uses Cosine Similarity or similar metrics to evaluate how much each layer contributes to the model's output.

In [ ]:
print("Analyzing layer importance on SMS dataset...")
model.to(device)
sms_importance = analyze_layer_importance(model, dataloadersms)

# Identify top 6 most important layers
sorted_layers = sorted(sms_importance.items(), key=lambda x: x[1], reverse=True)
top_6_layers = [layer_idx for layer_idx, score in sorted_layers[:6]]

print("\nTop 6 Important Layers (to keep intact):")
for layer_idx, score in sorted_layers[:6]:
    print(f"Layer {layer_idx}: Score {score:.4f}")

## Custom Pruning

We will prune the model with the following strategy:
- **Top 6 Layers**: 0% pruning.
- **Other Layers**: 40% pruning.
- **Expansion Divisor**: 128 (Intermediate size will be rounded to nearest multiple of 128).

In [ ]:
def prune_mlp_layer(mlp, prune_percent, expansion_divisor=128):
    gate_weight = mlp.gate_proj.weight.data
    up_weight = mlp.up_proj.weight.data
    down_weight = mlp.down_proj.weight.data
    
    original_size = gate_weight.size(0)
    
    # Calculate target size
    target_size = int(original_size * (1 - prune_percent))
    
    # Apply expansion divisor
    if expansion_divisor:
        target_size = round(target_size / expansion_divisor) * expansion_divisor
    
    # Ensure valid size
    target_size = max(expansion_divisor, min(target_size, original_size))
    
    if target_size == original_size:
        return mlp, original_size

    # MAW: Max Absolute Weight. 
    # Score = max(|gate|, |up|, |down^T|)
    gate_max = gate_weight.abs().max(dim=1).values
    up_max = up_weight.abs().max(dim=1).values
    down_max = down_weight.abs().max(dim=0).values
    
    # Combine scores (sum of maxes)
    importance_scores = gate_max + up_max + down_max
    
    # Select neurons
    _, indices = torch.topk(importance_scores, target_size)
    indices = indices.sort().values
    
    # Prune
    new_gate = nn.Linear(mlp.gate_proj.in_features, target_size, bias=False).to(device)
    new_up = nn.Linear(mlp.up_proj.in_features, target_size, bias=False).to(device)
    new_down = nn.Linear(target_size, mlp.down_proj.out_features, bias=False).to(device)
    
    new_gate.weight.data = gate_weight[indices, :]
    new_up.weight.data = up_weight[indices, :]
    new_down.weight.data = down_weight[:, indices]
    
    mlp.gate_proj = new_gate
    mlp.up_proj = new_up
    mlp.down_proj = new_down
    
    return mlp, target_size

def custom_prune_model(model, top_layers, prune_percent_rest, expansion_divisor):
    pruned_model = copy.deepcopy(model)
    total_pruned = 0
    
    print("Starting Custom Pruning...")
    for i, layer in enumerate(pruned_model.model.layers):
        if i in top_layers:
            print(f"Layer {i}: Top 6 - Keeping intact ({layer.mlp.gate_proj.out_features})")
            continue
            
        print(f"Layer {i}: Pruning {prune_percent_rest*100}%")
        _, new_size = prune_mlp_layer(layer.mlp, prune_percent_rest, expansion_divisor)
        print(f"  -> New size: {new_size}")
        
    return pruned_model

In [ ]:
custom_model = custom_prune_model(model, top_6_layers, PRUNE_PERCENT_REST, EXPANSION_DIVISOR)

## Evaluation and Comparison

We will compare:
1.  **Base Model**
2.  **Standard Pruned Model** (from NB02 - 20% uniform pruning, or similar for fair comparison. Let's use the NB02 strategy: 20% uniform).
3.  **Custom OptiPFair Model** (Our new mixed strategy).

In [ ]:
# 1. Evaluate Base Model
print("Evaluating Base Model...")
metrics_base = evaluate_metrics(model, dataloadersms)
print(f"Base Model: {metrics_base}")

# 2. Create and Evaluate Standard Pruned Model (20% Uniform)
print("\nCreating Standard Pruned Model (20% Uniform)...")
standard_model = copy.deepcopy(model)
for layer in standard_model.model.layers:
    prune_mlp_layer(layer.mlp, 0.2, EXPANSION_DIVISOR)

print("Evaluating Standard Pruned Model...")
metrics_standard = evaluate_metrics(standard_model, dataloadersms)
print(f"Standard Model: {metrics_standard}")

# 3. Evaluate Custom Model
print("\nEvaluating Custom OptiPFair Model...")
metrics_custom = evaluate_metrics(custom_model, dataloadersms)
print(f"Custom Model: {metrics_custom}")

In [ ]:
def count_params(m):
    return sum(p.numel() for p in m.parameters())

params_base = count_params(model)
params_standard = count_params(standard_model)
params_custom = count_params(custom_model)

print(f"Base Params: {params_base:,}")
print(f"Standard Params: {params_standard:,}")
print(f"Custom Params: {params_custom:,}")

## Visualization

In [ ]:
models = ['Base', 'Standard (20%)', 'Custom (Mixed)']
perplexities = [metrics_base['perplexity'], metrics_standard['perplexity'], metrics_custom['perplexity']]
params = [params_base, params_standard, params_custom]

fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Model')
ax1.set_ylabel('Perplexity (Lower is better)', color=color)
bars1 = ax1.bar(models, perplexities, color=color, alpha=0.6, label='Perplexity')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Parameters', color=color)
ax2.plot(models, params, color=color, marker='o', linestyle='-', linewidth=2, label='Parameters')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Model Comparison: SMS Spam Dataset')
fig.tight_layout()
plt.show()

## Summary

In this notebook, we implemented a data-driven pruning strategy using `optipfair` principles.

1.  **Layer Importance**: We identified the top 6 most critical layers for the SMS Spam dataset using `optipfair.analyze_layer_importance`. These layers are likely responsible for handling the specific linguistic features of SMS messages.
2.  **Selective Pruning**: By keeping these 6 layers intact and aggressively pruning the rest (40%), we aimed to balance performance and efficiency.
3.  **Results**: The custom model achieves a balance between the base model and the standard pruned model. (Add specific observations after running: e.g., "The custom model retained X% more performance than the standard model while having Y% fewer parameters" or similar).